https://github.com/vslobody/MUSIC/blob/master/music.py
https://github.com/dengjunquan/DoA-Estimation-MUSIC-ESPRIT


In [310]:
%matplotlib inline

In [311]:
import numpy as np
from scipy.constants import speed_of_light
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from typing import Tuple
import ipywidgets as widgets


In [312]:
FREQUENCY_DEVIATION = 250e3
BLUETOOTH_FREQUENCY = 2450e6  # Center frequency of Bluetooth channel in Hz
CTE_FREQUENCY = BLUETOOTH_FREQUENCY + FREQUENCY_DEVIATION
CTE_WAVELENGTH = speed_of_light / CTE_FREQUENCY  # Bluetooth wavelength in m
CTE_TIME = 160e-6  # Constant Tone Extension time
CTE_SAMPLES = 50
NUM_SOURCES = 1
NUM_ANGLES_TO_SEARCH = 360
MAX_SEARCH_ANGLE = 90


In [313]:
np.random.seed(0)
plt.style.use("seaborn")


In [314]:
def response_vec(
    num_antennas: int, incident_angle: float, antenna_spacing: float
) -> np.ndarray:
    """Expected antenna array response column vector of IQ values, not accounting for noise"""
    antennas = np.arange(0, num_antennas) * antenna_spacing
    steering = np.exp(
        2j * np.pi * antennas * np.sin(np.deg2rad(incident_angle)) / CTE_WAVELENGTH
    ).reshape((-1, 1))
    return steering


In [315]:
def sample_cov(antenna_response: np.ndarray, snr: float) -> np.ndarray:
    """Returns the sample covariance matrix for the given antenna response"""
    sample_times = np.linspace(0, CTE_TIME, CTE_SAMPLES)
    num_antennas = antenna_response.size

    cov = np.zeros(shape=(num_antennas, num_antennas), dtype=complex)
    for t in sample_times:
        iq_sample_at_t = antenna_response + np.sqrt(0.5 / snr) * np.random.randn(
            num_antennas
        )
        cov += iq_sample_at_t @ iq_sample_at_t.conjugate().T
    cov /= CTE_SAMPLES

    return cov


In [316]:
def music(
    covariance: np.ndarray, antenna_spacing: float
) -> Tuple[np.ndarray, np.ndarray]:
    """Perform the MUSIC algorithm for the given covariance matrix.
    Returns the angles searched, with their corresponding power"""
    num_antennas = covariance.shape[0]
    _, cov_eigvecs = np.linalg.eig(covariance)
    non_noise_cov_eigvecs = cov_eigvecs[:, NUM_SOURCES:num_antennas]
    search_angles = np.linspace(
        -MAX_SEARCH_ANGLE, MAX_SEARCH_ANGLE, NUM_ANGLES_TO_SEARCH
    )

    signal_powers = np.ndarray(shape=(NUM_ANGLES_TO_SEARCH,), dtype=complex)
    for idx, angle in enumerate(search_angles):
        steering_at_test_angle = response_vec(num_antennas, angle, antenna_spacing)
        signal_powers[idx] = (
            1
            / (
                steering_at_test_angle.conjugate().T
                @ non_noise_cov_eigvecs
                @ non_noise_cov_eigvecs.conjugate().T
                @ steering_at_test_angle
            )
        ).item()

    return search_angles, signal_powers


In [317]:
def draw_pseudo_spectrum(num_antennas, incident_angle, snr, antenna_spacing) -> None:
    """Draws the pseudo-spectrum for Bluetooth direction finding"""
    antenna_spacing /= 100  # convert to cm
    antenna_response = response_vec(num_antennas, incident_angle, antenna_spacing)
    sample_covariance = sample_cov(antenna_response, snr)
    search_angles, signal_powers = music(sample_covariance, antenna_spacing)
    signal_powers = signal_powers.real
    fig, ax = plt.subplots()

    peaks, _ = find_peaks(signal_powers, height=1)
    for peak in peaks:
        ax.axvline(search_angles[peak], ls="--", color="r")
    ax.semilogy(search_angles, signal_powers)


In [318]:
widget_style = {"description_width": "initial"}
num_antennas_input = widgets.BoundedIntText(value=5, min=2, description="Antennas:")
incident_input = widgets.IntSlider(
    value=60, min=-90, max=90, description="Incident angle (°):", style=widget_style
)
snr_input = widgets.FloatSlider(
    value=2.5, min=0.1, max=5, description="Signal-to-noise ratio", style=widget_style
)
antenna_spacing_input = widgets.FloatSlider(
    value=6, min=0.1, max=20, description="Antenna spacing (cm):", style=widget_style
)


interactive_plot = widgets.interactive(
    draw_pseudo_spectrum,
    num_antennas=num_antennas_input,
    incident_angle=incident_input,
    snr=snr_input,
    antenna_spacing=antenna_spacing_input,
)
interactive_plot.children[-1].layout.height = "500px"
interactive_plot


interactive(children=(BoundedIntText(value=5, description='Antennas:', min=2), IntSlider(value=60, description…